# Infer Request Share Input/Output Tensor Memory

OpenVINO provides `share_inputs` and `share_outputs` args in infer() method of InferRequest.

### Define benchmark function

In [ ]:
from time import time

import numpy as np
from ipywidgets import interact, widgets
from openvino.runtime import Core


def benchmark_model(infer_frames_func, batch_size):
    sec = 10
    count = 0
    start = time()
    inputs = np.random.randn(batch_size, 3, 224, 224)
    while time() - start < sec:
        infer_frames_func(inputs)
        count += batch_size

    print(f"FPS={count / sec:.2f}")


MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]

#### Choose Model

In [ ]:
model_name = "resnet50"
model_type = "int8"

models = ["resnet18", "resnet34", "resnet50", "resnet101", "resnet152"]


@interact(option=widgets.Dropdown(options=models, value=model_name, description="model name"))
def read_dropdown(option):
    global model_name
    model_name = option


model_types = ["fp32", "fp16", "int8"]


@interact(option=widgets.Dropdown(options=model_types, value=model_type, description="model type"))
def read_dropdown(option):
    global model_type
    model_type = option


model_path = f"models/{model_name}/{model_type}/model.xml"

### Define Infer Function

In [ ]:
def not_share_mem():
    model = Core().compile_model(model_path, "CPU")
    req = model.create_infer_request()

    def infer_frames(input_frames: np.ndarray):
        req.infer(input_frames)

    return infer_frames


def share_mem():
    model = Core().compile_model(model_path, "CPU")
    req = model.create_infer_request()

    def infer_frames(input_frames: np.ndarray):
        req.infer(input_frames, share_inputs=True, share_outputs=True)

    return infer_frames

### Benchmark

Choose batch size

In [ ]:
bs = 8


@interact(batch_size="8")
def read_input(batch_size):
    global bs
    bs = int(batch_size)

No Memory Share 

In [ ]:
benchmark_model(not_share_mem(), bs)

Enable Memory Share

In [ ]:
benchmark_model(share_mem(), bs)